#### Import Libraries

In [1]:
import os
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State
from plotly import tools

#### Set Log File

In [2]:
LOGFILE = 'log.csv'

#### Create Plotly Dash Application & Server

In [3]:
app = dash.Dash("UVA DataPalooza Plotly Dash Appication", static_folder='assets')
server = app.server

In [4]:
df = pd.read_csv('locations.csv')
mapbox_key = 'pk.eyJ1IjoibXJ1c2hpbiIsImEiOiJjam52MjA3Z2EwYjFlM2tyd25uZm9jN3NrIn0.xEAwa1BMFl_LCKbLpfhWGQ'

In [ ]:
def div_graph(name):
    return html.Div([
        html.Div(id=f'div-{name}-graph', className="ten columns"),
        html.Div([
            html.Div([
                html.P("Plot Display mode:", style={'font-weight': 'bold', 'margin-bottom': '0px'}),
                dcc.RadioItems(
                    options=[
                        {'label': ' Overlapping', 'value': 'overlap'},
                        {'label': ' Separate (Vertical)', 'value': 'separate_vertical'},
                        {'label': ' Separate (Horizontal)', 'value': 'separate_horizontal'}
                    ],
                    value='overlap', id=f'radio-display-mode-{name}'),
                html.Div(id=f'div-current-{name}-value')
            ]),
        ], className="three columns"),
    ], className="row")

In [ ]:
def update_graph(graph_id, graph_title, y_train_index, y_val_index, run_log_json, display_mode,  yaxis_title, xaxis_title):
    if run_log_json:
        layout = go.Layout(
            title=graph_title,
            margin=go.Margin(l=50, r=50, b=50, t=50),
            yaxis={'title': yaxis_title},
            xaxis={'title': xaxis_title}
        )
        
        run_log_df = pd.read_json(run_log_json, orient='split')
        step = run_log_df['epoch']
        y_train = run_log_df[y_train_index]
        y_val = run_log_df[y_val_index]
                    
        trace_train = go.Scatter(
            x=step,
            y=y_train,
            mode='lines',
            name='Training'
        )
        
        trace_val = go.Scatter(
            x=step,
            y=y_val,
            mode='lines',
            name='Validation'
        )
        
        if display_mode == 'separate_vertical':
            figure = tools.make_subplots(rows=2,
                                         cols=1,
                                         print_grid=False,
                                         shared_yaxes=True)
            
            figure.append_trace(trace_train, 1, 1)
            figure.append_trace(trace_val, 2, 1)
            figure['layout'].update(title=layout.title,
                                    margin=layout.margin,
                                    scene={'domain': {'x': (0., 0.5), 'y': (0.5,1)}})

        elif display_mode == 'separate_horizontal':
            figure = tools.make_subplots(rows=1,
                                         cols=2,
                                         shared_yaxes=True,
                                         print_grid=False)
            
            figure.append_trace(trace_train, 1, 1)
            figure.append_trace(trace_val, 1, 2)
            figure['layout'].update(title=layout.title,
                                    margin=layout.margin)

        elif display_mode == 'overlap':
            figure = go.Figure(
                data=[trace_train, trace_val],
                layout=layout
            )

        else:
            figure = None

        return dcc.Graph(figure=figure, id=graph_id)

    return dcc.Graph(id=graph_id)

In [ ]:
app.layout = html.Div([
    
    html.Div([
        html.H1('DataPalooza 2018'),
        html.Img(src="/assets/uva-datapalooza.png"),
        html.Img(src="/assets/bah-logo.png", style={"height" : "35%", "width" : "23%", "padding-top" : "3%", "padding-bottom" : "0%", "padding-left" : "0%", "padding-right" : "1%", "position" : "relative", "float" : "right"}),
        html.Br([]),
        html.H2('Plotly Dash Live Model Training Viewer')
    ], className="banner"),
    
    html.Br([]),
    
    html.Div([
        html.Div([
            dcc.Graph(id='map', figure={
                'data': [{
                    'lat': df['Latitude'],
                    'lon': df['Longitude'],
                    'marker': {'size': 8, 'opacity': 0.6},
                    'type': 'scattermapbox',
                    'text': df['Classification']
                }],
                'layout': {
                    'mapbox': {
                        'accesstoken': mapbox_key,
                        'bearing': 0,
                        'center': {'lat': 37.00900436, 'lon': -120.0736532},
                        'pitch': 0,
                        'zoom': 5
                    },
                'hovermode': 'closest',
                'margin': {'l': 0, 'r': 0, 'b': 0, 't': 0}
                }
            })
        ], className="twelve columns")
    ], className="row"),
    
    html.Br([]),
    
    html.Div([
        dcc.Dropdown(
            id='dropdown-interval-control',
            options=[
                {'label': 'No Updates', 'value': 'no'},
                {'label': 'Slow Updates', 'value': 'slow'},
                {'label': 'Regular Updates', 'value': 'regular'},
                {'label': 'Fast Updates', 'value': 'fast'}
            ],
            value='regular',
            className='twelve columns',
            clearable=False,
            searchable=False
        ),
        html.Div(id="div-epoch-display",className="two columns")
    ], id='div-interval-control', className='row'),
    
    dcc.Interval(id="interval-log-update", n_intervals=0),
    
    html.Div(id='run-log-storage', style={'display': 'none'}),
    
    html.Div([
        html.Div(div_graph('accuracy'), className='six columns'),
        html.Div(div_graph('cross-entropy'), className='six columns')
    ], className="row"),

], className="container")

In [ ]:
@app.callback(Output('interval-log-update', 'interval'), [Input('dropdown-interval-control', 'value')])
def update_interval_log_update(interval_rate):
    if interval_rate == 'fast':
        return 500
    elif interval_rate == 'regular':
        return 1000
    elif interval_rate == 'slow':
        return 5 * 1000
    elif interval_rate == 'no':
        return 24 * 60 * 60 * 1000

@app.callback(Output('run-log-storage', 'children'), [Input('interval-log-update', 'n_intervals')])
def get_run_log(_):
    names = ['epoch', 'train accuracy', 'train cross entropy', 'val accuracy', 'val cross entropy']
    try:
        run_log_df = pd.read_csv(LOGFILE, names=names)
        run_log_df = run_log_df[['epoch', 'train accuracy', 'val accuracy', 'train cross entropy', 'val cross entropy']]
        json = run_log_df.to_json(orient='split')
    except FileNotFoundError as error:
        print(error)
        print("Please verify if the csv file generated by your model is placed in the correct directory.")
        return None
    return json

@app.callback(Output('div-epoch-display', 'children'), [Input('run-log-storage', 'children')])
def update_div_step_display(run_log_json):
    if run_log_json:
        run_log_df = pd.read_json(run_log_json, orient='split')
        return html.H6(f"Epoch: {run_log_df['epoch'].iloc[-1]}", style={'margin-top': '3px'})


@app.callback(Output('div-accuracy-graph', 'children'), [Input('run-log-storage', 'children'), Input('radio-display-mode-accuracy', 'value')])
def update_accuracy_graph(run_log_json, display_mode):
    graph = update_graph('accuracy-graph', 'Prediction Accuracy', 'train accuracy', 'val accuracy', run_log_json, display_mode, 'Accuracy', 'Epoch')
    try:
        if display_mode in ['separate_horizontal', 'overlap']:
            graph.figure.layout.yaxis['range'] = [0, 1]
        else:
            graph.figure.layout.yaxis1['range'] = [0, 1]
            graph.figure.layout.yaxis2['range'] = [0, 1]
    except AttributeError:
        pass
    return [graph]

@app.callback(Output('div-cross-entropy-graph', 'children'), [Input('run-log-storage', 'children'), Input('radio-display-mode-cross-entropy', 'value')])
def update_cross_entropy_graph(run_log_json, display_mode):
    graph = update_graph('cross-entropy-graph', 'Binary Cross Entropy Loss', 'train cross entropy', 'val cross entropy', run_log_json, display_mode, 'Loss', 'Epoch')
    return [graph]

In [ ]:
external_css = [
    "https://cdnjs.cloudflare.com/ajax/libs/normalize/7.0.0/normalize.min.css",
    "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
    "https://codepen.io/mrushin/pen/yRKJQg.css"]

for css in external_css:
    app.css.append_css({"external_url": css})

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "UVA DataPalooza Plotly Dash Appication" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2018 10:37:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:27] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:28] "GET /assets/bah-logo.png HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:28] "GET /assets/uva-datapalooza.png HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:29] "POST /_dash-update-component HTTP/1.1" 20

127.0.0.1 - - [31/Oct/2018 10:37:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:37:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [31/Oct/2018 10:38:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [31/Oct/2018 10:38:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 10:38:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.

...

127.0.0.1 - - [31/Oct/2018 11:04:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [31/Oct/2018 11:04:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:04:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [31/Oct/2018 11:05:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [31/Oct/2018 11:05:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:05:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [31/Oct/2018 11:06:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:06:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2018 11:06:14] "POST /_dash-update-component HTTP/1.1" 200 -
